## PHASE ONE: Processing Image To Generate Embeddings

In [2]:
"""     
This script is designed to give you a quick crash course in Cassandra / Elastic search to aid in what you need to do here
"""
# ** Add these to all jupyter notebooks so that we dont need to keep reloading and running shit from scratch
%load_ext autoreload
%load_ext autotime
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime


In [28]:
milvus_utils.connect_to_milvus()
milvus_utils.clear_all_collections()
milvus_utils.create_face_public_keys_collection()

Connecting to Milvus at https://in03-71f797713109e68.serverless.gcp-us-west1.cloud.zilliz.com...
Successfully connected to Milvus.
Deleting collection 'face_public_keys'...
All collections have been deleted successfully.
Creating collection 'face_public_keys'...
Collection 'face_public_keys' created.
Creating index BIN_IVF_FLAT for field 'face_public_key_vector'...
Index created successfully.


<Collection>:
-------------
<name>: face_public_keys
<description>: Collection for storing face public keys and detailed metadata
<schema>: {'auto_id': True, 'description': 'Collection for storing face public keys and detailed metadata', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'face_public_key_vector', 'description': '', 'type': <DataType.BINARY_VECTOR: 100>, 'params': {'dim': 256}}, {'name': 'face_public_key', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 512}}, {'name': 'image_base_64_id', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 100}}, {'name': 'first_name', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 100}}, {'name': 'middle_name', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 100}}, {'name': 'surname', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 

In [4]:
import warnings
warnings.filterwarnings("ignore")

## Derive the BASE_DIR based on the current file location
import os
import sys
temp = os.getcwd()
vals = temp.split('/')
BASE_DIR = '/'.join(vals[:-1])
BASE_DIR = '%s/' % BASE_DIR
print(BASE_DIR)
sys.path.insert(0, BASE_DIR)

from bson import ObjectId

/Users/dr_d3mz/Library/Mobile Documents/com~apple~CloudDocs/utom_codebase/utom_client_projects/theaccubin/face_public_key/


In [5]:
import matplotlib.pyplot as plt
import face_utils
import geometrical_features
import face_embeddings
import public_key_gen
import milvus_utils 
import image_utils
import nin_metadata_insertion
import nin_metadata_retreival

I0000 00:00:1744594217.804253   37321 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 83.1), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1744594217.829391   37968 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744594217.840458   37963 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Initializing FaceNet embedder...
FaceNet embedder initialized successfully.
Connecting to Milvus at https://in03-71f797713109e68.serverless.gcp-us-west1.cloud.zilliz.com...
Successfully connected to Milvus.


In [6]:
image_paths = {
    'trump': os.path.join(BASE_DIR, 'image_data', 'trump.jpg'),
    'vance': os.path.join(BASE_DIR, 'image_data', 'vance.jpg'),
    'obama': os.path.join(BASE_DIR, 'image_data', 'obama.jpg'),
    'vance_2': os.path.join(BASE_DIR, 'image_data', 'vance_2.jpg'),
    'vance_3': os.path.join(BASE_DIR, 'image_data', 'vance_3.jpg'),
    'samuel_l_young': os.path.join(BASE_DIR, 'image_data', 'samuel_l_young.jpg'),
    'samuel_l_old': os.path.join(BASE_DIR, 'image_data', 'samuel_l_old.jpg'),
}



In [17]:
# """    
# Inserting Nin Image
# """
# # Example usage
# image_path = image_paths['trump']
# nin_input_params = {}
# nin_input_params['image_path'] = image_path
# nin_metadata_insertion.generate_and_insert_nin_input(nin_input_params)

"""   
Testing Endpoint
"""
import json
import requests

def test_generate_nin_input_endpoint(image_path):
    url = "http://65.21.243.182:5001/daccubin/generate-nin-input"
    nin_input_params = {
        "first_name": "Donald",
        "middle_name": "John",
        "surname": "Trump",
        "date_of_birth": "14 Jun 46",
        "issue_date": "20 Jan 17",
        "nationality": "USA",
        "sex": "M",
        "height": "190",
        "NIN": "1234567890"
    }

    with open(image_path, 'rb') as image_file:
        files = {'image': image_file}
        response = requests.post(url, files=files, data={'nin_input_params': json.dumps(nin_input_params)})

    if response.status_code == 200:
        print("Test passed. NIN input dictionary generated successfully.")
        print(response.json())
    else:
        print(f"Test failed with status code {response.status_code}.")
        print(response.json())

def test_find_closest_face_key_endpoint(image_path):
    url = "http://65.21.243.182:5001/daccubin/find-closest-face-key"
    
    with open(image_path, 'rb') as image_file:
        files = {'image': image_file}
        response = requests.post(url, files=files)

    if response.status_code == 200:
        print("Test passed. Closest face key found successfully.")
        print(response.json())
    elif response.status_code == 500:
        print("Test failed with status code 500.")
        print(response.json())
    else:
        print(f"Test failed with status code {response.status_code}.")
        print(response.json())




In [33]:
# Example usage
test_generate_nin_input_endpoint(image_paths['samuel_l_old'])

Test passed. NIN input dictionary generated successfully.
{'nin_input_dict': {'NIN': '2160221465', 'date_of_birth': '14 Jun 46', 'face_public_key': 'ec18516e475ce5a91a4908b14463fe028b1de2d20065d794eb8fc438081f1eae', 'first_name': 'Donald', 'height': '190', 'image_base_64_id': '67fc701ee03956fab851c1aa', 'issue_date': '20 Jan 17', 'middle_name': 'John', 'nationality': 'USA', 'sex': 'M', 'surname': 'Trump'}}


In [34]:
image_path = image_paths['samuel_l_old']
test_find_closest_face_key_endpoint(image_path)

Test passed. Closest face key found successfully.
{'NIN': '2160221465', 'distance': 0.0, 'input_base_64_string': '...', 'retrieved_base64_string': '...'}


In [27]:

# Example usage
image_path = image_paths['obama']
result = nin_metadata_retreival.find_closest_face_key_and_retrieve_data(image_path)
print(result.keys())
print()

Image shape: (2048, 1640, 3)
MediaPipe Face Detection initialized (model=1, confidence=0.6).


I0000 00:00:1744596664.512706   37321 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 83.1), renderer: Apple M1
W0000 00:00:1744596664.579385   95738 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Generating embedding for the input image...


W0000 00:00:1744596664.651050   37965 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Embedding generated successfully.
Generating projection matrix (256x517) with seed 42...
Projection matrix generated.

Searching 'face_public_keys' for key closest to: a89e456c0f... (top_k=1)
Search completed in 0.2513 seconds.
Found 1 potential match(es). Processing...
Closest match found: Distance=0.0, NIN=4797774116, MongoID=67fc6e2a6a6d9871b52b54e0
Retrieving image data from MongoDB using NIN: 4797774116...
dict_keys(['NIN', 'distance', 'input_base_64_string', 'retrieved_base64_string'])



In [ ]:


# Insert dummy data only if the collection is empty
if face_public_keys_collection.num_entities == 0:
        milvus_utils.insert_dummy_data(face_public_keys_collection, 5000)
else:
        print(f"Collection already has {face_public_keys_collection.num_entities} entities.")

In [ ]:
from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
    AnnSearchRequest, RRFRanker, WeightedRanker # For potential future hybrid search/ranking
)

test_query_key = milvus_utils.generate_dummy_key_hex()

"""   
load this in when the function is loaded, so that we dont have to do it at each function call
"""
# Ensure the collection is loaded before searching
if not utility.has_collection(collection_name):
    raise ValueError(f"Collection {collection_name} does not exist.")

print("Loading collection before search...")
face_public_keys_collection.load()
utility.wait_for_loading_complete(collection_name)
print("Collection loaded.")

In [ ]:
temp = milvus_utils.find_closest_key(face_public_keys_collection, test_query_key)

In [ ]:
milvus_utils.clear_all_collections()

In [ ]:
import time


# --- Main Execution ---




# Example Search: Generate a random key and find its nearest neighbor
# (In a real scenario, this would come from your application)

# Alternatively, to test with a known key (if dummy data was just inserted):
# test_query_key = binary_key_to_hex(entities[0][0]) # Get hex of first inserted key


# Ensure the collection is loaded before searching
if face_key_collection.partitions and not face_key_collection.partitions[0].is_loaded:
    print("Loading collection before search...")
    face_key_collection.load()
    utility.wait_for_loading_complete(COLLECTION_NAME)
    print("Collection loaded.")

metadata, dist = find_closest_key(face_key_collection, test_query_key)

# Example: Search for a key that *should* be in the database if dummy data was added
if face_key_collection.num_entities >= BATCH_SIZE:
        # Re-fetch an inserted key to test retrieval
        res = face_key_collection.query(expr=f"{FIELD_METADATA} == 'user_meta_000'", output_fields=[FIELD_VECTOR])
        if res:
            known_binary_vector = res[0][FIELD_VECTOR]
            known_hex_key = known_binary_vector.hex()
            print(f"\n--- Searching for a known key: {known_hex_key[:10]}... ---")
            metadata, dist = find_closest_key(face_key_collection, known_hex_key)
            # Expect distance to be 0 or very small if found correctly


except Exception as e:
print(f"An error occurred: {e}")
finally:
# Optional: Disconnect
# connections.disconnect("default")
# print("Disconnected from Milvus.")
pass # Keep connection open if running interactively or in a long-running app

In [ ]:
face_public_key

In [9]:


public_keys = {}

# for name, path in image_paths.items():
#     public_keys[name] = generate_face_public_key_from_image(path)

# print(public_keys)

In [ ]:
def compare_public_keys(public_keys):
    comparisons = {}
    keys = list(public_keys.keys())
    for i in range(len(keys)):
        for j in range(i + 1, len(keys)):
            key1 = keys[i]
            key2 = keys[j]
            print(f"Comparing {key1} and {key2}")
            distance = public_key_gen.compare_keys(public_keys[key1], public_keys[key2])
            comparisons[f"{key1} vs {key2}"] = distance
            print(f"Distance between {key1} and {key2}: {distance}")
    return comparisons

distances = compare_public_keys(public_keys)
print(distances)

In [ ]:
distances

In [ ]:
"""   

"""